In [58]:
data_path = "../datasource/sqlResult_1558435.csv"
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
import re
import numpy as np 
from sklearn.neural_network import MLPClassifier

1. import data
2. tf-idf 表征
3. 模型训练

1. 构建一个机器学习模型，判断这个文章是不是新华社的
2. 当这个模型的acc 大于 0.8778， recall， precision，f1等值都较高的时候
3. 用该模型 判断一篇文章是否是新华社的文章，如果判断出来是新华社的，但是，它的source并不是新华社的，那么，我们就说，这个文章是抄袭的新华社的文章

In [3]:
df = pd.read_csv(data_path, encoding= "GB18030")


In [4]:
df.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


## test tf-idf

In [5]:
test_corpus = []
for i in range(10):
    page = " ".join(jieba.cut(df['content'][i]))
    test_corpus.append(page)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(test_corpus)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.774 seconds.
Prefix dict has been built succesfully.


- [ ] tf-idf 的 vector 实际上是该词在各篇文章中的tf-idf值组成的向量

In [6]:
X.shape

(10, 2044)

In [7]:
len(vectorizer.get_feature_names())

2044

## TF-IDF


In [50]:
non_null_df = df[~df['content'].isna()]

In [55]:
non_null_df["source"].unique()
non_null_df.count()

id         87054
author     76925
source     87052
content    87054
feature    87054
title      87054
url        87054
dtype: int64

In [52]:
uncut_corpus = non_null_df["content"].tolist()
cut_corpus = list(map(lambda x :" ".join(jieba.cut(x)),uncut_corpus))

In [53]:
vectorlizer = TfidfVectorizer()
Y = non_null_df["source"]=="新华社"
X = vectorlizer.fit_transform(cut_corpus)

In [70]:
X.shape

(87054, 243613)

## train_test_split

In [140]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.33,random_state=42)

## pca
尝试使用PCA降维，对于sparsed matrix 需要使用 `TruncatedSVD` 。但降维到100效果不明显。提升维度，内存报错。

应该加入分批训练。


In [142]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(200)
tsvd.fit(X_train)
cumsum = np.cumsum(tsvd.explained_variance_ratio_)

In [143]:
cumsum[-1]

0.2312351413766137

In [144]:
pca_X_train = tsvd.transform(X_train)
pca_X_test = tsvd.transform(X_test)

## incremental PCA  XXXXXX

必须是密集矩阵。

- [-]

In [145]:
from sklearn.decomposition import IncrementalPCA

n_batches =10000

inc_pca = IncrementalPCA(n_components=5)
inc_pca.partial_fit(X[:10,:].toarray())
reduced = inc_pca.transform(X.toarray())

MemoryError: 

## deep neural network

In [146]:
dnn = MLPClassifier((200,200,100,50,5))

In [147]:
dnn.fit(pca_X_train,Y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(200, 200, 100, 50, 5),
              learning_rate='constant', learning_rate_init=0.001, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [148]:
Y_train_hat = dnn.predict(pca_X_train)

In [150]:
Y_test_hat = dnn.predict(pca_X_test)

In [152]:
(Y_test!=Y_test_hat).sum()/Y_test.sum()

0.036408420890582306

In [165]:
def tf_idf(series):
    corpus = series.tolist()
    x= list(map(lambda x :" ".join(jieba.cut(x)),corpus))
    sparse_vector = vectorlizer.transform(x)
    pca_vector = tsvd.transform(sparse_vector)
    return pca_vector


def classifier(series):
    transformed_x = tf_idf(series)
    y_hat = dnn.predict(transformed_x)
    return y_hat

def score(y,y_hat):
    tp = ((y==y_hat)[y==True]).sum()
    tn = ((y==y_hat)[y==False]).sum()
    fp = ((y!=y_hat)[y==True]).sum()
    fn = ((y!=y_hat)[y==False]).sum()
    print("confusion matrix")
    print("true","false")
    print("positive",tp,fp)
    print("negative",tn,fn)
    precision = tp / (tp+tn)
    recall = tn/(tn+fn)
    return (precision,recall)

def classify_score(x,y):
    y_hat = classifier(x)
    p,r = score(y,y_hat)
    print("precision",p)
    print("recall",p)

In [160]:
score(Y_test,Y_test_hat)

confusion matrix
true false
positive 25606 377
negative 2176 569


(0.9216759052624001, 0.7927140255009107)

In [162]:
sample = df['content'][:10]

In [163]:
sample

0    此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1    骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2    此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...
3      这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n
4    （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...
5    　　受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金...
6    虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是...
7    　　沙漠雄鹰：震荡有利消化套牢筹码\r\n　　周四开盘上证50在银行券商大蓝筹带动下一度涨近...
8    （原标题：武汉警方一下子抓了808人，还都是俊男靓女！原来他们每天偷偷摸摸干这事！）\r\n...
9    　　6月21日，A股纳入MSCI指数尘埃落定，但当天被寄予厚望的券商股并未扛起反弹大旗。22...
Name: content, dtype: object

In [166]:
y_sample_hat = classifier(sample)

In [167]:
print(y_sample_hat)

[False False False  True  True False False False False False]


In [168]:
print(df['source'][:10])

0      快科技@http://www.kkj.cn/
1      快科技@http://www.kkj.cn/
2      快科技@http://www.kkj.cn/
3                         新华社
4                       深圳大件事
5                   中国证券报?中证网
6    威锋网@http://www.feng.com/
7                   中国证券报?中证网
8                   荆楚网-楚天都市报
9                   中国证券报?中证网
Name: source, dtype: object
